In [16]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
# import seaborn as sns
from scipy import stats
import re
# import scipy.stats as stats

In [17]:
source_file = 'wage2.csv'
url = 'https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/'
df = pd.read_csv(url + source_file)
df

,wage,hours,IQ,KWW,educ,exper,tenure,age,married,black,south,urban,sibs,brthord,meduc,feduc,lwage
0,769,40,93,35,12,11,2,31,1,0,0,1,1,2.0,8.0,8.0,6.645091
1,808,50,119,41,18,11,16,37,1,0,0,1,1,NaN,14.0,14.0,6.694562
2,825,40,108,46,14,11,9,33,1,0,0,1,1,2.0,14.0,14.0,6.715384
3,650,40,96,32,12,13,7,32,1,0,0,1,4,3.0,12.0,12.0,6.476973
4,562,40,74,27,11,14,5,34,1,0,0,1,10,6.0,6.0,11.0,6.331502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,520,40,79,28,16,6,1,30,1,1,1,0,0,1.0,11.0,NaN,6.253829
931,1202,40,102,32,13,10,3,31,1,0,1,1,7,7.0,8.0,6.0,7.091742
932,538,45,77,22,12,12,10,28,1,1,1,0,9,NaN,7.0,NaN,6.287858
933,873,44,109,25,12,12,12,28,1,0,1,0,1,1.0,NaN,11.0,6.771935


## 1.5 wage equation #2

Для набора данных `wage2` рассмотрим линейную регрессию ***log(wage)*** на ***age, IQ, south***: исходную и со структурными сдвигами относительно **места жительства**.

In [18]:
# formula = 'np.log(wage) ~ age + IQ + south + married'
formula = 'np.log(wage) ~ age + IQ + south'
Sleep_1 = smf.ols(formula = formula, data = df).fit()
# formula = (f'np.log(wage) ~ age + IQ + south + married'
#            f'+ urban + age:urban+IQ:urban+south:urban + married:urban')
formula = (f'np.log(wage) ~ age + IQ + south'
           f'+ urban + age:urban+IQ:urban+south:urban')
Sleep_2 = smf.ols(formula = formula, data = df).fit()
print(
    summary_col(
        [Sleep_1, Sleep_2],
        float_format='%.3f',
        stars=True
    )
)



               np.log(wage) I np.log(wage) II
---------------------------------------------
IQ             0.008***       0.008***       
               (0.001)        (0.002)        
IQ:urban                      -0.000         
                              (0.002)        
Intercept      5.214***       4.999***       
               (0.168)        (0.312)        
R-squared      0.146          0.182          
R-squared Adj. 0.143          0.176          
age            0.023***       0.024***       
               (0.004)        (0.007)        
age:urban                     -0.001         
                              (0.009)        
south          -0.113***      -0.014         
               (0.028)        (0.052)        
south:urban                   -0.117*        
                              (0.061)        
urban                         0.237          
                              (0.368)        
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [19]:
rest = Sleep_1.rsquared
# rest = 0.188
print(f'rest = {rest:3.3f}')
unrest = Sleep_2.rsquared
# unrest = 0.205
print(f'unrest = {unrest:3.3f}')

rest = 0.146
unrest = 0.182


In [25]:
F_obs = (unrest - rest)/(1-unrest)*(934-8)/4
print(f'F_obs = {F_obs:.2f}')

F_obs = 10.24


#### Уровень значимости 10%.

In [24]:
alpha = 0.1 # Уровень значимости
F_cr = stats.f.ppf(1-alpha,4,934-8)
print(f'F_cr = {F_cr:.2f}')


F_cr = 1.95


In [22]:
prompt = 'Значимы ли структурные сдивиги? '
reply = 'Значимы' if F_obs > F_cr else 'Незначимы'
print(f'{prompt} {reply}')

Значимы ли структурные сдивиги?  Значимы
